<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/2%EC%9B%94%206%EC%9D%BC/0205_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 불러오기

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 46.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

/usr/local/lib/python3.8/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [4]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# 2. 데이터 불러오기 및 분리하기


In [21]:
df = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/model_data.parquet')

- 구매했으니 1로 둠

In [22]:
df['values']=1

- groupby로 만들어서 train, test customer_id 비율을 맞춰줬음
  - 비율이 안맞아서 검증셋에 없는 customer_id가 있지만 그래도 균등하게 나눠진듯

In [23]:
test = df.groupby('c_idx').sample(frac=0.2,random_state=42)
train = df.drop(test.index)

In [24]:
print(test['c_idx'].nunique(),'\n',train['c_idx'].nunique())

37704 
 50698


In [25]:
print(train.shape, test.shape)

(1005166, 6) (248627, 6)


In [26]:
user_items = sparse.csr_matrix((train['values'],(train['c_idx'], train['p_idx'])))
item_users = sparse.csr_matrix((train['values'],(train['p_idx'],train['c_idx'])))

In [28]:
#산거
train_df = train.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()
train_df

,c_idx,p_idx
0,0,"[6337, 8625, 3202, 3323, 21844, 5246, 21377, 1..."
1,1,"[19759, 22592, 5571, 12489, 25493, 13859]"
2,2,"[2336, 9053, 13775, 10912, 19757, 13006]"
3,3,[14068]
4,4,"[13807, 15504, 13706, 14526, 20247, 23329, 29779]"
...,...,...
50693,50700,"[11382, 23942, 21796, 29029, 25744, 23241, 297..."
50694,50701,"[18765, 11976, 16954, 2578, 57, 20124, 3562, 6..."
50695,50702,[10623]
50696,50703,"[4622, 11566, 8822, 30886, 30757, 21690, 2971,..."


In [27]:
# 검증셋 만들기
test_df = test.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()
test_df

,c_idx,p_idx
0,0,"[11860, 27541, 14880, 9968, 4662, 12501, 11680..."
1,1,"[6328, 6461]"
2,2,"[16457, 9568]"
3,4,"[27200, 28617]"
4,6,"[18336, 2008, 21018, 15434, 16156, 31063, 247,..."
...,...,...
37699,50698,"[22224, 28354]"
37700,50699,[23766]
37701,50700,"[7153, 26461, 27675]"
37702,50701,"[2180, 11685, 20149, 14136, 571, 12309, 2072, ..."


- 0인 값이 몇 %?

In [29]:
matrix_size = user_items.shape[0]* user_items.shape[1]
num_purchases = len(user_items.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.93649644127566

# 3. implicit 라이브러리 사용

- **recommend(user_id, user_items[user_id], filter_already_liked_items=True/False, N=k)**
  - filter_already_liked_items=True 면 훈련데이터에 없는 상품 추천
  - filter_alreadt_liked_items=False 면 훈련데이터에 있는 거 포함해서 추천

- **explain(user_id, user_items, item_id)**<br>
사용자가 항목을 좋아하는 이유에 대한 설명 제공<br>
  - total_score: user/item에 대한 예측 점수
  - total_contributions : user/item에 대한 상위 N(item_id, score) 기여 목록
  - user_weights : user의 분해된 표현

- **similar_items(item_id)**
  - 유사한 상품 
- **similar_users(user_id)**
  - 유사한 유저 

In [30]:
ALS = implicit.als.AlternatingLeastSquares(
    factors=200, regularization=40, alpha=40,
    iterations=20, random_state=42, calculate_training_loss=True
)

In [31]:
ALS.fit(user_items)

  0%|          | 0/20 [00:00<?, ?it/s]

In [32]:
user_vecs = ALS.user_factors
item_vecs = ALS.item_factors

In [33]:
print(user_vecs.shape,'\n',item_vecs.shape)

(50705, 200) 
 (31121, 200)


# 4. 추천

In [34]:
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product.parquet')

In [38]:
# 중복값제거 
product_c =product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)

In [42]:
def recommendation(customer_idx,k):
  item = ALS.recommend(customer_idx,user_items[customer_idx],k)[0]
  recommend=pd.DataFrame(columns=product_c.columns)
  for i in item:
    item_re= product_c[product_c['p_idx'].isin([i])]
    recommend=pd.concat([recommend,item_re])
  return recommend

In [43]:
recommendation(2212,10)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
953,48911,Women,Accessories,Jewellery,Pendant,Silver,Summer,2013,Casual,Lucera Women Silver Pendant with Chain,16318
1424,47794,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Murcia Women Brown Handbag,17291
2102,54174,Women,Accessories,Jewellery,Necklace and Chains,Gold,Summer,2012,Casual,Femella Golden Necklace,9396
4803,59524,Women,Accessories,Jewellery,Earrings,Silver,Summer,2015,Casual,Lucera Silver Earrings,16294
1606,11922,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Franco Leone Men Formal Black Formal Shoes,10714
2170,44923,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Jack & Jones Men Blue Jeans,12986
7727,59380,Women,Apparel,Topwear,Kurtas,Blue,Summer,2012,Ethnic,Vishudh Women Blue Printed Kurta,29504
2057,38836,Women,Apparel,Dress,Dresses,Blue,Summer,2012,Casual,Latin Quarters Women Blue Dress,14455
175,35322,Women,Accessories,Jewellery,Earrings,Silver,Winter,2016,Ethnic,Fabindia Women Anusuya Silver Earrings,8542
9013,44001,Men,Personal Care,Fragrance,Perfume and Body Mist,Navy Blue,Spring,2017,Casual,Police Men Pure Perfume,21072


# 5. precision/recall@k

In [66]:
# als추천과 test셋에 있는 p_idx 비교
def precision_recall(user_id):
  
  count=0
  user_buy = test[test['c_idx']==user_id].p_idx
  recommendations_t = ALS.recommend(user_id, user_items[user_id],filter_already_liked_items=True,N=25)
  for i in user_buy:
    for j in recommendations_t[0]:
      if i==j:
        count+=1
  
  precision = count/len(recommendations_t[0])

  try: # userbuy가 없는 데이터셋이 존재
    recall = count/len(user_buy)
    return precision, recall

  except:
    pass
  

In [67]:
precision_recall(26569)

(0.04, 0.008)

In [68]:
precision_recall(4582)

In [72]:
precision_recall(639)

(0.0, 0.0)

- 흐음

In [73]:
def precision(user_id):
  count=0
  recommendations_f = ALS.recommend(user_id, user_items[user_id],filter_already_liked_items=False,N=25)
  recommendations_t = ALS.recommend(user_id, user_items[user_id],filter_already_liked_items=True,N=25)
  for i in recommendations_f[0]:
    for j in recommendations_t[0]:
      if i==j:
        count+=1
  
  precision = count/len(recommendations_t[0])  
  return precision

In [74]:
precision(4569)

0.52

In [77]:
precision(26569)

1.0

In [76]:
precision(89)

0.68

In [81]:
precision(47)

0.48

# 6. 고객 세분화

In [83]:
df_group=df.groupby(['c_idx'], as_index = False).count()
upper_19 = list(df_group.query("values >= 19").index)
lower_19 = list(df_group.query("values < 19").index)

In [1]:
def recommendations(customer_id,k):
  '''
  if : 물건 산 횟수가 19회 이상인 경우 -> ALS.recommend() 이용
  elif : 물건 산 횟수가 19회 이하인 경우 -> ALS.similar_items() 이용 
         젤 최근에 산 물건의 인덱스를 similar_items()에 넣기
  else : 물건을 사지 않은 경우 -> 젤 많이 팔린 상품 추천
  
  CB는 언제 사용하지?! CB는 문제가 물건이 너무 많아서 램이 터짐 그래서 15000개까지만 됨..
  '''
  if customer_id in upper_19:
    recommend_t = ALS.recommend(customer_id, user_items[customer_id],filter_already_liked_items=True,N=k)

  elif customer_id in lower_19:
    pass
    
  
  else:
    pass